In [1]:
import sys

python_path = ['/home/claudio/.local/lib64/python3.6/site-packages','/home/sebastian/.local/lib64/python3.6/site-packages','/home/sebastian/.local/lib/python3.6/site-packages','/home/sebastian/.local/lib/python3.6/site-packages/opencv-4.1.0-py3.6.egg/cv2',
               '/home/sebastian/.local/lib/python3.6/site-packages/cv2','/home/sebastian/dat/onnx/onnx-tensorrt/third_party/onnx']
sys.path.extend(python_path)
from torchnlp.word_to_vector import FastText
%load_ext autoreload
%autoreload 2
import pandas as pd
from utils import *
from zero_shot_model import *
from PIL import Image
from PIL import ImageFile

In [48]:
device = 'cuda:1'

model = ZeroshotModel.load_model(device, 'models/zero_shot_model_seba_semantics/')

embedding_matrix = get_category_embedding_matrix().to(device)


semantic_embeddings = pd.read_pickle('semantic_embeddings.pkl')
categories = semantic_embeddings.Class.values
mask = np.isin(categories, test_categories)
mask = np.argwhere(mask)

df = pd.read_pickle('tags_onehot_corrected.pkl')
    
files = pd.read_pickle('good_files.pkl')[0].values

df = df[df.file.isin(files)]
    
indices = []
for i in range(df.shape[0]):
    row = df.iloc[i,:-1].values
    if np.sum(row) == 1:
        indices.append(i)
            
unitag = df.iloc[indices]

Agrego Centers


In [49]:
model.eval_mode()

In [50]:
def accuracy(category):
    
    samples = unitag[unitag[category] == 1].file.values
    
    #results = [0]*len(categories)
    
    results= {}
    
    
    for i in range(samples.shape[0]):
        
        img = Image.open('/data/hotel_images/'+samples[i])
        
        img_tensor = image_transformations_with_normalization(img).repeat(1,1,1,1)
    
        p , _, _ = model(img_tensor.to(device))
        
        sum_logits = torch.zeros(p.shape[1], embedding_matrix.shape[1]).to(device)
        for i in range(4):
            sum_logits += torch.mm(p[i], embedding_matrix.to(device))

        sum_logits = torch.nn.functional.softmax(sum_logits, dim = 1)

        sum_logits = sum_logits.T[mask.ravel()].T

        indices_pred = torch.topk(sum_logits, k = 1, dim = 1)[1].detach().cpu().numpy().squeeze()
        
        #results[indices_pred] += 1
        
        cat = categories[mask[indices_pred]][0]
        
        if cat  in results:
            results[cat] = results[cat] + 1
        else:
            results[cat] = 1
            
        
    return results

In [51]:
res_kitchen = accuracy('kitchen')
res_health_club = accuracy('health_club')
res_restaurant = accuracy('restaurant')
res_natural_view = accuracy('natural_view')

#results = np.array([res_kitchen, res_health_club, res_restaurant, res_natural_view])

#results = pd.DataFrame(data=results, index=['kitchen', 'health_club','restaurant', 'natural_view'], columns=categories)

/home/claudio/.local/lib/python3.6/site-packages/torch/nn/functional.py:2705: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "


In [52]:
res_kitchen

{'health_club': 297, 'kitchen': 427, 'natural_view': 54}

In [53]:
res_natural_view

{'natural_view': 1212, 'health_club': 4, 'kitchen': 2}

In [54]:
res_restaurant

{'natural_view': 366, 'health_club': 439, 'kitchen': 167}

In [55]:
res_health_club

{'health_club': 118, 'natural_view': 195, 'kitchen': 12}

In [27]:
df = pd.read_pickle('tags_onehot_corrected.pkl')

In [28]:
files = pd.read_pickle('good_files.pkl')

In [31]:
df = df[df.file.isin(files[0].values)]

In [30]:
files[0].values

array(['2884bcd3-73f6-4694-882d-970f309b8d15.jpg',
       '69d44bea-3940-46f1-bf0a-a345ab5b7ca5.jpg',
       'b04904d9-70b8-40e2-8949-fe49530461e7.jpg', ...,
       'sun_bejyseuksiozrxjl.jpg', 'sun_aemqrnupbojbmaql.jpg',
       'sun_dqliiydyhpnjwrfz.jpg'], dtype=object)

In [32]:
df.shape

(34942, 32)

In [33]:
df.to_pickle('multitag_dataset.pkl')